[View in Colaboratory](https://colab.research.google.com/github/MChamith/SpatioAE/blob/master/trainae.ipynb)

In [1]:
!git clone https://github.com/MChamith/SpatioAE

Cloning into 'SpatioAE'...
remote: Counting objects: 7091, done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 7091 (delta 6), reused 0 (delta 0), pack-reused 7076
Receiving objects: 100% (7091/7091), 219.96 MiB | 26.30 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Checking out files: 100% (7204/7204), done.


In [2]:

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Activation, BatchNormalization
from keras.models import Model
from keras import backend as K
import cv2
import numpy as np
import os
from tqdm import tqdm
import fnmatch

TRAIN_DIR ='SpatioAE/Train'
TEST_DIR = 'SpatioAE/Test'
IMG_SIZE = 128



def create_training_set():
  train_data = []
  matches = []
  for root, dirnames, filenames in tqdm(os.walk(TRAIN_DIR)):
    for filename in fnmatch.filter(filenames, "*.tif"):
      path = os.path.join(root, filename)
      img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE) )
      img = np.expand_dims(img, axis = 2)
      train_data.append(np.array(img))
          
  np.save('training_data.npy' , train_data)
  
  return train_data

      
    
    
#   for img in tqdm(os.listdir(TRAIN_DIR)):
#     path = os.path.join(TRAIN_DIR, img)
#     img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE) )
#     img = np.expand_dims(img, axis = 2)
#     train_data.append(np.array(img))
    
    
#   np.save('training_data.npy' , train_data)
  
#   return train_data


def create_test_set():
  test_data = []
  for img in tqdm(os.listdir(TEST_DIR)):
    path = os.path.join(TEST_DIR, img)
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE) )
    img = np.expand_dims(img, axis = 2)
    test_data.append(np.array(img))
    
    
  np.save('test_data.npy' , test_data)
  return test_data

Using TensorFlow backend.


In [0]:

def model_autoencoder():
  
  input_img  = Input(shape = (128,128,1))  #channel last
  
  x = Conv2D(filters = 32, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(input_img)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
  print("layer one conv " + str(x.shape))
  
  x = Conv2D(filters = 48, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
  print("layer two conv " + str(x.shape))
  
 
  x = Conv2D(filters = 64, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
  print("layer three conv " + str(x.shape))
  
  x = Conv2D(filters = 64, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  encoded = MaxPooling2D(pool_size = (1,1), strides = (1,1))(x)
  print("layer four conv " + str(encoded.shape))
  
  
  ## decoder branch
  
  x = Conv2D(filters = 48, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(encoded)
  x = UpSampling2D(size = (2,2))(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  print("layer one deconv " + str(x.shape))
 
  x = Conv2D(filters = 32, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = UpSampling2D(size = (2,2))(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  print("layer two deconv " + str(x.shape))
  
  x = Conv2D(filters = 32, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = UpSampling2D(size = (2,2))(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  print("layer three deconv " + str(x.shape))
  decoded = Conv2D(filters = 1, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'sigmoid')(x)
  print("output" + str(decoded.shape))

  
  
  autoencoder = Model(input_img, decoded)
  
  return autoencoder
  

In [0]:
get_data = create_training_set()
input_imgs = np.load('training_data.npy')
input_imgs = input_imgs.astype('float32') / 255.
print(input_imgs.shape)
K.clear_session()
autoencoder = model_autoencoder()
autoencoder.compile(optimizer = 'adam' , loss = 'mean_squared_error')

autoencoder.fit(input_imgs,input_imgs,  epochs =50, batch_size= 100)
  
    

In [51]:
import matplotlib.pyplot as plt

print(input_imgs[1])

[[[0.35686275]
  [0.31764707]
  [0.4117647 ]
  ...
  [0.33333334]
  [0.16862746]
  [0.23529412]]

 [[0.3019608 ]
  [0.30980393]
  [0.43137255]
  ...
  [0.34509805]
  [0.21568628]
  [0.25490198]]

 [[0.34509805]
  [0.38039216]
  [0.34509805]
  ...
  [0.34901962]
  [0.28235295]
  [0.32156864]]

 ...

 [[0.41960785]
  [0.42745098]
  [0.43137255]
  ...
  [0.64705884]
  [0.6509804 ]
  [0.68235296]]

 [[0.4392157 ]
  [0.4509804 ]
  [0.45882353]
  ...
  [0.65882355]
  [0.64705884]
  [0.6784314 ]]

 [[0.45490196]
  [0.47058824]
  [0.47058824]
  ...
  [0.6784314 ]
  [0.65882355]
  [0.6862745 ]]]


In [0]:
get_test_data = create_test_set()
test_imgs = np.load('test_data.npy')
test_imgs = input_imgs.astype('float32') / 255.
print(input_imgs.shape)

decode_imgs = autoencoder.predict(test_imgs)